## CS 156 HW 1
### Ankush Hommerich-Dutt

In [126]:
import random
import timeit

class Coin:
    def __init__(self):
        self.outcomes = []
    
    def flip(self):
        rand = random.random()
        if rand < 0.5:
            self.outcomes.append('t')
        else:
            self.outcomes.append('h')
    
def main(n):
    v_1 = [0] * n
    v_rand = [0] * n
    v_min = [0] * n
    coins = [Coin() for j in range(1000)]
    for i in range(n):
        min_heads = 11
        for coin in coins:
            for j in range(10):
                coin.flip()
            num_heads = coin.outcomes.count('h')
            if num_heads < min_heads:
                min_heads = num_heads
        
        v_min[i] = min_heads
        
        v_1[i] = coins[0].outcomes.count('h')
        v_rand[i] = coins[random.randint(0,999)].outcomes.count('h')
        
        for coin in coins:
            coin.outcomes = []  
    
    v_1_avg = (sum(v_1) / n) / 10
    v_rand_avg = (sum(v_rand) / n) / 10
    v_min_avg = (sum(v_min) / n) / 10
    
    print("Running ", n, " iterations of 1000 coins flipped 10 times.")
    print("The average value of v_min is: ", v_min_avg)
    print("The average value of v_1 is: ", v_1_avg)
    print("The average value of v_rand is: ", v_rand_avg)

main(100000)            
 

Running  100000  iterations of 1000 coins flipped 10 times.
The average value of v_min is:  0.03783
The average value of v_1 is:  0.500831
The average value of v_rand is:  0.49919


1. **[b]**

The average value calculated for $\nu_{\text{min}}$ is  0.037 which is closest to 0.01 out of the answer choices. The code to get this result is above.

2. **[d]**

Hoeffding inequality: $\mathbb P[|\nu - \mu| > \epsilon] \leq 2e^{-2\epsilon^2N}$

$\mu$ for a coin flip is 0.5. 

For $\nu_{\text{min}}$, |\nu - \mu| = |0.0378 - 0.5| = 0.4622 

If we plug in 0.4622 as epsilon into the Hoeffding inequality, we get $e^{-2000(0.4622)^2} = 2.78 \times 10^{-186}$ which is a very low probability (next to none), and so this distribution does not satisfy the Hoeffding inequality. This distribution for $\nu$ will never happen if our $\mu$ was actually 0.5 

For $\nu_{\text{1}}$, |\nu - \mu| = |0.5008 - 0.5| = 0.0008

If we plug in 0.008 as epsilon into the Hoeffding inequality, we get $e^{-2000(0.0008)^2} = .9987$ which is a very high probability. This distribution for $\nu$ is very likely to occur given that our $\mu$ is 0.5, and so we can safely say that the $c_{\text{1}}$ distribution satisfies Hoeffding.

For $\nu_{\text{rand}}$, |\nu - \mu| = |0.4992 - 0.5| = 0.0008

If we plug in 0.008 as epsilon into the Hoeffding inequality, we get $e^{-2000(0.0008)^2} = .9987$ which is a very high probability. This distribution for $\nu$ is very likely to occur given that our $\mu$ is 0.5, and so we can safely say that the $c_{\text{rand}}$ distribution satisfies Hoeffding.


3. **[e]** 

The probability of $h$ approximating $y$ incorrectly given the noisy $f$ is the probability of $h$ approximating $f$ correctly when $f$ gets $y$ wrong, plus the probability of $h$ approximating $f$ incorrectly when $f$ gets $y$ right. 
Expressing this mathematically in terms of the variables defined in the problem, this is the same as:

$[(1 - \lambda) * (1 - \mu)] + [\lambda * \mu]$

4. **[b]**

If we expand out the expression from the previous answer of the probability of error:

$[(1 - \lambda) * (1 - \mu)] + [\lambda * \mu] = 1 +\mu\lambda - \lambda - \mu + \mu\lambda = 1 + 2\mu\lambda - \lambda - \mu = 1 + \mu(2\lambda - 1) - \lambda$

If $\lambda = 0.5$, then the $\mu$ term in the error expression goes to 0, and our probability of error is $1 - 0.5 = 0.5$, regardless of what $\mu$ is.

In [132]:
import numpy as np
from numpy import linalg as LA

class Point:
    def __init__(self, x, y):
        self.val = np.array([1, x, y])
        
    def outcome(self, line):
        target = line.evaluate(self.val[1])
        if self.val[2] > target:
            return 1
        return -1
        
class Line:
    def __init__(self, point1, point2):
        self.slope = (point2.val[2] - point1.val[2]) / (point2.val[1] - point1.val[1])
        self.intercept = (self.slope * (-point1.val[1])) + point1.val[2]
        
    def evaluate(self, x):
        return (self.slope * x) + self.intercept
    
        
def random_point():
    return Point(random.uniform(-1,1), random.uniform(-1,1))

def target_function():
    point1 = random_point()
    point2 = random_point()
    return Line(point1, point2)

class Weight_vector:
    def __init__(self):
        self.vec = np.array([0,0,0]) 
    
    def update(self, y, point):
        self.vec[0] += y*point.val[0]
        self.vec[1] += y*point.val[1]
        self.vec[2] += y*point.val[2]

def iteration(outcomes, weight_vec, target):
    mis = [i[0] for i in outcomes if np.sign(weight_vec.vec.dot(i[0].val)) != i[1]]
    if len(mis) == 0:
        return True
    bad_point = mis[random.randint(0,len(mis) - 1)]
    weight_vec.update(bad_point.outcome(target), bad_point)
    
    return False

    
def main(n):
    target = target_function()
    data = [random_point() for x in range(n)]
    outcomes =  list(map(lambda x: (x,x.outcome(target)), data))

    X = np.zeros((n, 3))
    y = np.zeros(n)
    for i in range(n):
        X[i,:] = outcomes[i][0].val
        y[i] = outcomes[i][1]
        
    w = Weight_vector()
        
    w.vec = (LA.inv(X.T@X)@(X.T))@y   
    #w = LA.pinv(X)@y
    
    mis = 0
    for i in range(n):
        if np.sign(w.vec.dot(data[i].val)) != outcomes[i][1]:
            mis += 1
        
    E_in = mis / n
    
    new_data = [random_point() for x in range(1000)]
    new_outcomes =  list(map(lambda x: (x, x.outcome(target)), new_data))
    mis = [1 for i in new_outcomes if np.sign(w.vec.dot(i[0].val)) != i[1]]
    prob = len(mis) / 1000
    
    converge = False 
    num_iterations = 0
    while converge != True:
        converge = iteration(outcomes, w, target)
        num_iterations += 1
    
    return E_in, prob, num_iterations

test = [main(100) for i in range(1000)]
test2 = [main(10) for i in range(1000)]
print("N = 100: ")
print("E_in: ", sum(i for i, _, _ in test) / 1000)
print("E_out: ", sum(i for _, i, _ in test) / 1000)
print ("N = 10: ")
print("Average number of iterations for PLA to converge: ", sum(i for _, _, i in test2) / 1000)


N = 100: 
E_in:  0.04007000000000012
E_out:  0.05046499999999996
N = 10: 
Average number of iterations for PLA to converge:  6.229


5. **[c]** 

The E_in of 0.04 is closest to 0.01

6. **[d]**

The E_out of 0.0504 is closest to 0.1

7. **[a]**

6.229 iterations is closest to 1 iteration (closer than 15)

In [127]:
import math
import numpy as np
from numpy import linalg as LA

class Point:
    def __init__(self, x, y):
        self.val = np.array([1, x, y])
        
class Non_linear:
    def __init__(self, x, y):
        self.val = np.array([1, x, y, x*y, math.pow(x, 2), math.pow(y, 2)])
        
class Test_function:
    def __init__(self, c1, c2, c3, c4, c5):
        self.val = [0, c1, c2, c3, c4, c5]
        
def evaluate_test(non, func):
    return np.sign(-1 - (func.val[1]*non.val[1]) + (func.val[2]*non.val[2]) + \
                   (func.val[3]*non.val[3]) + (func.val[4]*non.val[4]) + (func.val[5]*non.val[5]))  
    
        
def random_point():
    return Point(random.uniform(-1,1), random.uniform(-1,1))


def evaluate(point):
    output = np.sign(math.pow(point.val[1], 2) + math.pow(point.val[2], 2) - 0.6)
    if random.random() < 0.1:
        return -output
    return output

    
def main(n):
    data = [random_point() for x in range(n)]
    outcomes =  list(map(lambda x: (x, evaluate(x)), data))               

    X = np.zeros((n, 3))
    y = np.zeros(n)
    for i in range(n):
        X[i,:] = outcomes[i][0].val
        y[i] = outcomes[i][1]
        
        
    w = (LA.inv(X.T@X)@(X.T))@y   
    #w = LA.pinv(X)@y
    
    mis = 0
    for i in range(n):
        if np.sign(w.dot(data[i].val)) != outcomes[i][1]:
            mis += 1
        
    E_in = mis / n
    
    
    data2 = [Non_linear(x.val[1], x.val[2]) for x in data]             

    X_2 = np.zeros((n, 6))
    y_2 = np.zeros(n)
    for i in range(n):
        X_2[i,:] = data2[i].val
        y_2[i] = outcomes[i][1]
        
    #w_2 = (LA.inv(X_2.T@X_2)@(X_2.T))@y_2 
    w_2 = LA.pinv(X_2)@y_2

    g1 = Test_function(0.05, 0.08, 0.13, 1.5, 1.5)
    g2 = Test_function(0.05, 0.08, 0.13, 1.5, 15)
    g3 = Test_function(0.05, 0.08, 0.13, 15, 1.5)
    g4 = Test_function(1.5, 0.08, 0.13, 0.05, 0.05)
    g5 = Test_function(0.05, 0.08, 1.5, 0.15, 0.15)
    
    g1_mis = 0
    g2_mis = 0
    g3_mis = 0
    g4_mis = 0
    g5_mis = 0
    
    for i in range(n):
        if evaluate_test(data2[i], g1) != np.sign(w_2.dot(data2[i].val)):
            g1_mis += 1
        if evaluate_test(data2[i], g2) != np.sign(w_2.dot(data2[i].val)):
            g2_mis += 1
        if evaluate_test(data2[i], g3) != np.sign(w_2.dot(data2[i].val)):
            g3_mis += 1
        if evaluate_test(data2[i], g4) != np.sign(w_2.dot(data2[i].val)):
            g4_mis += 1
        if evaluate_test(data2[i], g5) != np.sign(w_2.dot(data2[i].val)):
            g5_mis += 1  
            
    new_points = [random_point() for x in range(1000)]
    new_outcomes =  list(map(lambda x: (x, evaluate(x)), new_points))   
    new_data = [Non_linear(x.val[1], x.val[2]) for x in new_points] 
    new_mis = 0
    for i in range(1000):
        if np.sign(w_2.dot(new_data[i].val)) != new_outcomes[i][1]:
            new_mis += 1
            
    E_out = new_mis / 1000
    
    
    
    return E_in, g1_mis / n, g2_mis / n, g3_mis / n, g4_mis / n, g5_mis / n, E_out 

test = [main(1000) for i in range(1000)]
print("E_in: ", sum(i for i, _, _, _, _, _, _ in test) / 1000)
print("chance of g1 agreeing with h: ", 1 - (sum(i for _, i, _, _, _, _, _ in test) / 1000))
print("chance of g2 agreeing with h: ", 1 - (sum(i for _, _, i, _, _, _, _ in test) / 1000))
print("chance of g3 agreeing with h: ", 1 - (sum(i for _, _, _, i, _, _, _ in test) / 1000))
print("chance of g4 agreeing with h: ", 1 - (sum(i for _, _, _, _, i, _, _ in test) / 1000))
print("chance of g5 agreeing with h: ", 1 - (sum(i for _, _, _, _, _, i, _ in test) / 1000))
print("E_out: ", sum(i for _, _, _, _, _, _, i in test) / 1000)

E_in:  0.5037600000000001
chance of g1 agreeing with h:  0.9617010000000001
chance of g2 agreeing with h:  0.6621519999999995
chance of g3 agreeing with h:  0.6620479999999993
chance of g4 agreeing with h:  0.6327049999999997
chance of g5 agreeing with h:  0.5612019999999996
E_out:  0.1268460000000001


8. **[d]**

E_in is 0.504, which is closest to 0.5

9. **[a]** 

Out of the 5 hypotheses, g1 agrees with w 96% of the time, which is much higher than any of the other hypotheses. Therefore, it is closest to w.

10. **[b]** 

E_out is 0.127, which is closest to 0.1